# Classifying Book Author's gender by name (approximately)

Used dataset: https://www.kaggle.com/datasets/gracehephzibahm/gender-by-name/data

In [1]:
%cd ..

C:\Users\jplak\projects\women_books


C:\Users\jplak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\Local\pypoetry\Cache\virtualenvs\women-books-tYhTzxuG-py3.10\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## 0. Import Data

In [51]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [3]:
df_names = pd.read_csv("./data/gender_by_name_dataset.csv")

In [5]:
df_books = pd.read_parquet("./datasets/2024-01-19_books.parquet")

## 1. Name dataset EDA and preprocessing

In [8]:
df_names

,Name,Gender,Count,Probability
0,James,M,5304407,1.451679e-02
1,John,M,5260831,1.439753e-02
2,Robert,M,4970386,1.360266e-02
3,Michael,M,4579950,1.253414e-02
4,William,M,4226608,1.156713e-02
...,...,...,...,...
147264,Zylenn,M,1,2.736740e-09
147265,Zymeon,M,1,2.736740e-09
147266,Zyndel,M,1,2.736740e-09
147267,Zyshan,M,1,2.736740e-09


In [9]:
df_names["Gender"].value_counts()

Gender
F    89749
M    57520
Name: count, dtype: int64

In [10]:
df_female_names = df_names[df_names["Gender"] == "F"].copy()

In [28]:
df_female_names[
    (~df_female_names["Name"].str.isalpha())
    & (~df_female_names["Name"].str.contains("-"))
]

,Name,Gender,Count,Probability
25258,Smt.,F,282,7.717610e-07
27085,Chlo…,F,247,6.759750e-07
31858,Am…Lie,F,180,4.926130e-07
42282,Esm…,F,94,2.572540e-07
55930,Ku.,F,43,1.176800e-07
...,...,...,...,...
133622,Yoshoda@,F,1,2.736740e-09
133673,Za'Quiesha,F,1,2.736740e-09
133782,Ze'Eva,F,1,2.736740e-09
133888,Zo'E,F,1,2.736740e-09


In [31]:
df_female_names[
    df_female_names["Name"].str.endswith(".")
]

,Name,Gender,Count,Probability
25258,Smt.,F,282,7.717610e-07
55930,Ku.,F,43,1.176800e-07
65877,Km.,F,25,6.841850e-08
93539,Smts.,F,6,1.642040e-08
101918,Miss.,F,5,1.368370e-08
101985,Ms.,F,5,1.368370e-08
110755,Mrs.,F,4,1.094700e-08
111446,Kr.,F,3,8.210220e-09
113109,Kum.,F,2,5.473480e-09
117455,B.,F,1,2.736740e-09


In [61]:
df_names[df_names["Name"] == "Mark"]

,Name,Gender,Count,Probability
23,Mark,M,1410637,0.003861
4216,Mark,F,4668,0.000013


In [62]:
df_names[df_names["Count"] < 5_000]

,Name,Gender,Count,Probability
4052,Brandyn,M,4999,1.368100e-05
4053,Konnor,M,4995,1.367000e-05
4054,Rubie,F,4994,1.366730e-05
4055,Arther,M,4989,1.365360e-05
4056,Fran,F,4986,1.364540e-05
...,...,...,...,...
147264,Zylenn,M,1,2.736740e-09
147265,Zymeon,M,1,2.736740e-09
147266,Zyndel,M,1,2.736740e-09
147267,Zyshan,M,1,2.736740e-09


In [65]:
# df_names = df_names[df_names["Count"] 

In [68]:
name_and_count: pd.Series = df_names.groupby("Name")["Count"].count()
single_sex_names_mask = name_and_count == 1

In [69]:
single_sex_names = name_and_count[single_sex_names_mask]

In [76]:
df_names_single_sex = df_names[df_names["Name"].isin(single_sex_names.index)].copy()
df_names_mixed_sex = df_names[~df_names["Name"].isin(single_sex_names.index)].copy()

In [77]:
df_names_mixed_sex

,Name,Gender,Count,Probability
0,James,M,5304407,1.451679e-02
1,John,M,5260831,1.439753e-02
2,Robert,M,4970386,1.360266e-02
3,Michael,M,4579950,1.253414e-02
4,William,M,4226608,1.156713e-02
...,...,...,...,...
147222,Ziyuan,M,1,2.736740e-09
147224,Zlata,M,1,2.736740e-09
147243,Zowie,M,1,2.736740e-09
147246,Zozo,M,1,2.736740e-09


## 2. Classification through exact match

In [14]:
female_names_set = set(df_female_names["Name"])

In [29]:
df_books["name_chunks"] = df_books["Book-Author"].str.split()

In [49]:
def filter_chunks(chunk_list):
    return [
        el 
        for el in chunk_list 
        if not (
            el.endswith(".")
            and len(el) <= 2
        )
    ]

In [50]:
filter_chunks(["A.", "Miss."])

['Miss.']

In [52]:
df_books["name_chunks"] = df_books["name_chunks"].progress_apply(filter_chunks)

100%|██████████████████████████████████████████████████████████████████████| 271379/271379 [00:00<00:00, 551779.48it/s]


In [56]:
df_books["name_chunks"]

0                  [Mark, Morford]
1         [Richard, Bruce, Wright]
2                  [Carlo, D'Este]
3             [Gina, Bari, Kolata]
4                         [Barber]
                    ...           
271374           [Paula, Danziger]
271375               [Teri, Sloat]
271376         [Christine, Wicker]
271377                     [Plato]
271378       [Christopher, Biffle]
Name: name_chunks, Length: 271379, dtype: object

In [57]:
def check_in_set(chunks: list[str], names=female_names_set) -> bool:
    return any((name in names for name in chunks))

In [58]:
df_books["matched_female"] = df_books["name_chunks"].progress_apply(check_in_set)

100%|█████████████████████████████████████████████████████████████████████| 271379/271379 [00:00<00:00, 1032242.00it/s]


In [59]:
df_books[df_books["matched_female"]]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-M,name_chunks,matched_female
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,"[Mark, Morford]",True
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,"[Richard, Bruce, Wright]",True
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,"[Carlo, D'Este]",True
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,"[Gina, Bari, Kolata]",True
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,[Barber],True
...,...,...,...,...,...,...,...,...
271373,449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993,Ballantine Books,http://images.amazon.com/images/P/0449906736.0...,"[Robin, Wright]",True
271374,440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,"[Paula, Danziger]",True
271375,525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,"[Teri, Sloat]",True
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,"[Christine, Wicker]",True


In [60]:
"Mark" in female_names_set

True